# Preparation stuff

## Connect to Drive

In [23]:
connect_to_drive = False

In [24]:
#Run command and authorize by popup --> other window
if connect_to_drive:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)

## Install packages

In [25]:
if connect_to_drive:
    #Install FS code
    #!pip install  --upgrade --no-deps --force-reinstall git+https://github.com/federicosiciliano/easy_lightning.git@fedsic
    !pip install  --upgrade --no-deps --force-reinstall git+https://github.com/PokeResearchLab/easy_lightning.git

    !pip install pytorch_lightning

## IMPORTS

In [26]:
#Put all imports here
import numpy as np
import matplotlib.pyplot as plt
#from copy import deepcopy
#import pickle
import os
import sys
#import cv2
import torch
import csv
from copy import deepcopy

## Define paths

In [27]:
#every path should start from the project folder:
project_folder = "../"
if connect_to_drive:
    project_folder = "/content/gdrive/Shareddrives/<SharedDriveName>" #Name of SharedDrive folder
    #project_folder = "/content/gdrive/MyDrive/<MyDriveName>" #Name of MyDrive folder

#Config folder should contain hyperparameters configurations
cfg_folder = os.path.join(project_folder,"cfg")

#Data folder should contain raw and preprocessed data
data_folder = os.path.join(project_folder,"data")
raw_data_folder = os.path.join(data_folder,"raw")
processed_data_folder = os.path.join(data_folder,"processed")

#Source folder should contain all the (essential) source code
source_folder = os.path.join(project_folder,"src")

#The out folder should contain all outputs: models, results, plots, etc.
out_folder = os.path.join(project_folder,"out")
img_folder = os.path.join(out_folder,"img")

## Import own code

In [28]:
#To import from src:

#attach the source folder to the start of sys.path
sys.path.insert(0, project_folder)
os.environ['PYTHONPATH'] = project_folder #for raytune workers

#import from src directory
from src.module import *

import easy_exp, easy_rec, easy_torch #easy_data

# MAIN

## Train

### Data

In [29]:
cfg = easy_exp.cfg.load_configuration("config_rec")

In [30]:
exp_id = 'Cxjc96JKlpQDLgf1'
exp_json_path = os.path.join(out_folder,"exp",cfg["__exp__"]["name"],f"{exp_id}.json")
# load json
with open(exp_json_path, "r") as f:
    exp_cfg = json.load(f)

In [31]:
# merge the two configurations
cfg = easy_exp.cfg.merge_dicts(cfg, exp_cfg, preference=1, merge_lists=True)

In [32]:
cfg["model.trainer_params"]["logger"]["params"].pop("version")

0

In [33]:
# def check_dict_diff(d1,d2):
#     for k,v in d1.items():
#         if k in d2 and d2[k]!=v:
#             if isinstance(v,dict):
#                 check_dict_diff(v,d2[k])
#             else:
#                 print(f"Diff in {k}: {cfg[k]} vs {v}")
#             #cfg[k] = v

In [34]:
#---> for _ in cfg.sweep("data_params.name"):
#---> for _ in cfg.sweep("model.rec_model.emb_size"):

In [35]:
exp_found, experiment_id = easy_exp.exp.get_set_experiment_id(cfg)
print("Experiment already found:", exp_found, "----> The experiment id is:", experiment_id)

if not exp_found: raise Exception("Experiment not found")
if experiment_id != exp_id: raise Exception("Experiment id mismatch")

# if exp_found and if_exp_found == "skip":
#     #print("Skipping experiment")
#     return

# Save experiment (done here cause Early stopping with Tune schedulers may not run anything after training)
# easy_exp.exp.save_experiment(cfg)

Experiment already found: True ----> The experiment id is: Cxjc96JKlpQDLgf1


In [36]:
data, maps = easy_rec.preparation.prepare_rec_data(cfg)

Ratings data already exists. Skip pre-processing
Filtering by minimum number of users per item: 10
Filtering by minimum number of items per user: 10
Densifying index
Splitting: leave_n_out


In [37]:
loaders = easy_rec.preparation.prepare_rec_dataloaders(cfg, data, maps)

In [38]:
main_module = easy_rec.preparation.prepare_rec_model(cfg, maps)

Seed set to 42
/home/caldia/recsys-svd/rec_svd/lib/python3.12/site-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(


### Decomposition

In [39]:
prepare_embeddings_based_on_init(cfg, main_module, processed_data_folder)

SVD Warning: unused kwargs dict_keys(['svd_params', 'cutoff', 'freeze_emb', 'invert_order', 'u', 'knee'])
Embedding Warning: unused kwargs dict_keys(['method', 'svd_params', 'use_diag', 'u', 's', 'v'])
NO CUTOFF


In [40]:
trainer = easy_torch.preparation.complete_prepare_trainer(cfg, experiment_id, additional_module=easy_rec)#, raytune=raytune)

metrics_list = cfg["model"]["metrics"].copy()
cfg["model"]["metrics"] = {}
for metric_name in metrics_list:
    cfg["model"]["metrics"][metric_name] = {'batch_metric' : True}
del cfg["model"]["metrics"]["MAP"]
del cfg["model"]["metrics"]["F1"]

model, model_params = easy_torch.preparation.complete_prepare_model(cfg, main_module, additional_module=easy_rec, return_model_params=True)

Seed set to 42
Seed set to 42
Seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42
Seed set to 42


In [41]:
model_path = os.path.join(out_folder, "models", cfg["__exp__"]["name"], experiment_id, "last.ckpt")
model = easy_torch.process.load_model(cfg["model"], model_path, main_module=main_module, map_location=torch.device('cuda'))
model.custom_log = lambda x, y: None
model.optimizer = model_params["optimizer"]
model.loss = model_params["loss"]
model.metrics = model_params["metrics"]

In [42]:
easy_torch.process.test_model(trainer, model, loaders)

Seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    0.3519110679626465     │
└───────────────────────────┴───────────────────────────┘